In [1]:
%matplotlib widget
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from util import get_path, get_dates_datetime, get_dirname

from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips
from subprocess import call


***Chose plate number and directory of interest where the folders with images are***

In [17]:
plate = 14
from directory import directory, run_parallel, find_state
# directory = "/scratch/shared/mrozemul/Fiji.app/"
listdir = os.listdir(directory) 
list_dir_interest = [name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
dates_datetime = get_dates_datetime(directory,plate)
len(list_dir_interest)

154

***Chose timestep to begin with and folder to end with***

In [18]:
begin = 0
end = 153
print('begin =', dates_datetime[begin], '\n  end =', dates_datetime[end])

begin = 2020-09-22 13:17:00 
  end = 2020-10-25 23:13:00


In [19]:
find_state(plate,begin,end)

([(datetime.datetime(2020, 9, 22, 17, 17), 1),
  (datetime.datetime(2020, 9, 22, 21, 17), 2),
  (datetime.datetime(2020, 9, 23, 1, 17), 3),
  (datetime.datetime(2020, 9, 23, 5, 17), 4),
  (datetime.datetime(2020, 9, 23, 9, 17), 5),
  (datetime.datetime(2020, 9, 23, 13, 17), 6),
  (datetime.datetime(2020, 9, 23, 17, 17), 7),
  (datetime.datetime(2020, 9, 23, 21, 17), 8),
  (datetime.datetime(2020, 9, 24, 1, 18), 9),
  (datetime.datetime(2020, 9, 24, 5, 18), 10),
  (datetime.datetime(2020, 9, 24, 9, 18), 11),
  (datetime.datetime(2020, 9, 28, 15, 44), 12),
  (datetime.datetime(2020, 9, 28, 19, 44), 13),
  (datetime.datetime(2020, 9, 28, 23, 44), 14),
  (datetime.datetime(2020, 9, 29, 3, 44), 15),
  (datetime.datetime(2020, 9, 29, 7, 44), 16),
  (datetime.datetime(2020, 9, 29, 14, 43), 17),
  (datetime.datetime(2020, 9, 29, 18, 42), 18),
  (datetime.datetime(2020, 9, 29, 22, 43), 19),
  (datetime.datetime(2020, 9, 30, 18, 43), 20),
  (datetime.datetime(2020, 9, 30, 22, 43), 21),
  (dateti

In [16]:
dates_datetime_chosen = dates_datetime[begin:end+1]
dates = dates_datetime_chosen
for date in dates:
    directory_name = get_dirname(date, plate)
    path_snap = directory+directory_name
    call(f'rm -r {path_snap}',shell=True)

In [15]:
dates

[datetime.datetime(2020, 10, 13, 6, 21)]

In [14]:
path_snap

'/scratch/shared/AMF914/Fiji.app/20201013_0621_Plate14'

***Compress raw image***

In [65]:
num_parallel = 4
time = '2:00'
args = [plate]
run_parallel('compress_image.py',args, begin, end, num_parallel, time, 'compress')

***Skeletonization***
- Only 4 skeletonization processes can be run in parallel on one node

In [66]:
num_parallel = 4
time = '3:00:00'
low = 30
high = 80
extend = 30
args=[plate, low, high, extend]
run_parallel('extract_skel_surf.py',args,begin,end,num_parallel,time,'skeletonization')

***Check Skeletonization***
- The first cell loads the skeleton and the compressed raw image
- The second cell shows the overlap of the skeleton and the raw image

In [142]:
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

- Chose start and finish to display the overlap of the skeleton and the raw image : no more than 10 at a time or jupyter will crash.
- For display purpose, everything is compressed so connectivity may be lost
- colors are just a feature of compression

In [143]:
plt.close('all')
start = 0
finish = start + 2
for i in range(start,finish):
    plot_t_tp1([], [], None, None, skels[i], ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Check specific image***
- If something wrong is noticed in one of the skeletons one can chose to look closer at one of the images and the skeletonization process
- chose ***i*** equal to the timestep where something wrong has been noticed 

In [10]:
#chose i equal to the timestep where something wrong has been noticed 
i = 84

dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime

date =dates [i]
directory_name = get_dirname(date,plate)
path_snap=directory+directory_name
path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
try:
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
except:
    print('error_name')
    path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]
dim = (int(np.max(ys)-np.min(ys))+4096,int(np.max(xs)-np.min(xs))+4096)
ims = []
for name in tileconfig[0]:
    imname = '/Img/'+name.split('/')[-1]
    ims.append(imageio.imread(directory+directory_name+imname))


In [11]:
tileconfig

,0,1,2
0,/scratch/shared/AMF914/Fiji.app/20201013_0621_...,NaN,"(0.0, 0.0)"


- Chose a x,y position where you want to see how the skeletonization process went (x is the scale on the left on the images and y is the bottom scale)
- You can chose to display different part of the filter par commenting/uncommenting

In [75]:
### plt.close('all')

#chose a spot where to look closer at
linex  = 3700
liney = 3200

shape = (3000,4096)
linex *= 5
liney *= 5
for index,im in enumerate(ims):
    boundaries = int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))
    if boundaries[1]<=linex< boundaries[1]+shape[0] and boundaries[0]<=liney< boundaries[0]+shape[1]:
        print(index)
        im_cropped = im
        im_blurred =cv2.blur(im_cropped, (200, 200))
        im_back_rem = (im_cropped+1)/(im_blurred+1)*120
        im_back_rem[im_back_rem>=130]=130
        # # im_back_rem = im_cropped*1.0
        # # # im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
        frangised = frangi(im_back_rem,sigmas=range(1,20,4))*255
        # # frangised = cv2.normalize(frangised, None, 0, 255, cv2.NORM_MINMAX)
        hessian = hessian_matrix_det(im_back_rem,sigma = 20)
        blur_hessian = cv2.blur(abs(hessian), (20, 20))
    #     transformed = (frangised+cv2.normalize(blur_hessian, None, 0, 255, cv2.NORM_MINMAX)-im_back_rem+120)*(im_blurred>=35)
    #     transformed = (frangised+cv2.normalize(abs(hessian), None, 0, 255, cv2.NORM_MINMAX)-im_back_rem+120)*(im_blurred>=35)
        transformed = (frangised-im_back_rem+120)*(im_blurred>=35)
        low = 40
        high = 80
        lowt = (transformed > low).astype(int)
        hight = (transformed > high).astype(int)
        hyst = filters.apply_hysteresis_threshold(transformed, low, high)
        kernel = np.ones((3,3),np.uint8)
        dilation = cv2.dilate(hyst.astype(np.uint8) * 255,kernel,iterations = 1)
        for i in range(3):
            dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
            dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
        dilated = dilation>0
        
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(dilated.astype(np.uint8), connectivity=8)
        #connectedComponentswithStats yields every seperated component with information on each of them, such as size
        #the following part is just taking out the background which is also considered a component, but most of the time we don't want that.
        sizes = stats[1:, -1]; nb_components = nb_components - 1

        # minimum size of particles we want to keep (number of pixels)
        #here, it's a fixed value, but you can set it as you want, eg the mean of the sizes or whatever
        min_size = 4000  

        #your answer image
        img2 = np.zeros((dilated.shape))
        #for every component in the image, you keep it only if it's above min_size
        for i in range(0, nb_components):
            if sizes[i] >= min_size:
                img2[output == i + 1] = 1
        skeletonized = cv2.ximgproc.thinning(np.array(255*img2,dtype=np.uint8))
        nx_g = generate_nx_graph(from_sparse_to_graph(scipy.sparse.dok_matrix(skeletonized)))
        g,pos= nx_g
        tips = [node for node in g.nodes if g.degree(node)==1]
        dilated_bis = np.copy(img2)
        for tip in tips:
            branch = np.array(orient(g.get_edge_data(*list(g.edges(tip))[0])['pixel_list'],pos[tip]))
            orientation = branch[0]-branch[min(branch.shape[0]-1,20)]
            orientation = orientation/(np.linalg.norm(orientation))
            window = 20
            x,y = pos[tip][0],pos[tip][1]
            if x-window>=0 and x+window< dilated.shape[0] and y-window>=0 and y+window< dilated.shape[1]:
                shape_tip = dilated[x-window:x+window,y-window:y+window]
                dist = 20
                for i in range(dist):
                    pixel = (pos[tip]+orientation*i).astype(int)
                    xp,yp = pixel[0],pixel[1]
                    if xp-window>=0 and xp+window< dilated.shape[0] and yp-window>=0 and yp+window< dilated.shape[1]:
                        dilated_bis[xp-window:xp+window,yp-window:yp+window]+=shape_tip
        dilation = cv2.dilate(dilated_bis.astype(np.uint8) * 255,kernel,iterations = 2)
        for i in range(3):
            dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 2)
            dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 2)
        skeletonized = cv2.ximgproc.thinning(np.array(255*(dilation>0),dtype=np.uint8))
        print('Raw image')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(im)
        print('Raw image with background removed')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(im_back_rem)
        print('frangised image')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(frangised)
        print('final transformed image')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(transformed)
        print('threhsolded image')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(dilated>0)
        print('threhsolded image')
        fig=plt.figure(figsize=(10,9))
        
        ax = fig.add_subplot(111)
        ax.imshow(img2>0)
        print('extended tips')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(dilated_bis)
        print('final_skel')
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(cv2.normalize(abs(hessian), None, 0, 255, cv2.NORM_MINMAX)-255*dilated)
        plot_t_tp1([],[],None,None,skeletonized,im_back_rem)

81
Raw image


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Raw image with background removed


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

frangised image


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

extended tips


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

final_skel


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Mask baits and border of the petri dish***

In [186]:
num_parallel = 4
time = '5:00'
thresh = 30
args=[plate,thresh]
run_parallel('mask_skel.py',args,begin,end,num_parallel,time,'mask')

***Check Masking***

In [8]:
begin = 83
end = 85
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = dates_datetime_chosen
skels = []
ims = []
masks = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap = directory + directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_masked_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    mask_info = read_mat(path_snap+'/Analysis/mask.mat')
    mask = mask_info['mask']
    masks.append(mask)
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [9]:
plt.close('all')
start = 0
finish = start +2
for i in range(start,finish):
    plot_t_tp1([], [], None, None, masks[i], ims[i])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Prune Graph***

In [13]:
num_parallel = 20
time = '30:00'
threshold = 1
args = [plate,threshold]
run_parallel('prune_skel.py',args,begin,end,num_parallel,time,'prune_graph')

***Check Pruned Graphs***

In [152]:
begin = 20
end = 40
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_pruned_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

In [155]:
plt.close('all')
start = 5
finish = start + 2
for i in range(start,finish):
    plot_t_tp1([],[],None,None,skels[i],ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
plt.close('all')
kernel = np.ones((5,5),np.uint8)
for i in range(len(compressed)-1):
    plot_t_tp1([],[],None,None,cv2.dilate(compressed[i].astype(np.uint8),kernel,iterations = 2),cv2.dilate(compressed[i+1].astype(np.uint8),kernel,iterations = 2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Realign***

In [6]:
num_parallel = 8
time = '2:00:00'
thresh = 100000
args=[plate,thresh]
run_parallel('final_alignment.py',args,begin,end,num_parallel,time,'realign')

***Check Alignment***

In [21]:
plt.close('all')
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = dates_datetime_chosen
dilateds=[]
skels = []
skel_docs = []
Rs=[]
ts=[]
for date in dates[1:]:
    directory_name = get_dirname(date,plate)
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_pruned.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
    transform = sio.loadmat(path_snap+'/Analysis/transform.mat')
    R,t = transform['R'],transform['t']
    Rs.append(R)
    ts.append(t)


In [22]:
plt.close('all')
start = 0
for j in range(start,start + 20):
    print(dates[j],j+begin)
    skeleton1,skeleton2 = skel_docs[j],skel_docs[j+1]
    R,t = Rs[j+1],ts[j+1]
    skelet_pos = np.array(list(skeleton1.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    X = np.transpose(skelet_pos[samples,:])
    skelet_pos = np.array(list(skeleton2.keys()))
    samples = np.random.choice(skelet_pos.shape[0],20000)
    Y = np.transpose(skelet_pos[samples,:])
    fig=plt.figure(figsize=(10,9))
    ax = fig.add_subplot(111)
    Yrep=np.transpose(np.transpose(np.dot(R,X))+t)
#     ax.scatter(np.transpose(X)[:,0],np.transpose(X)[:,1])
    ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
    ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

2020-11-26 06:06:00 40


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2020-11-26 10:06:00 41


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2020-11-26 14:12:00 42


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2020-11-29 06:16:00 58


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

2020-11-29 14:16:00 59


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Create realigned Skeleton***

In [5]:
num_parallel = 10
time = '30:00'
args = [plate, begin, end]
run_parallel('realign_surf.py', args, begin, end, num_parallel, time, 'create_realign')

***Check Fine Alignment***

In [15]:
dates_datetime = get_dates_datetime(directory, plate)
dates_datetime_chosen = dates_datetime[begin:end+1]
dates = dates_datetime_chosen
skels = []
ims = []
kernel = np.ones((5, 5), np.uint8)
itera = 1
for date in dates:
    directory_name = get_dirname(date,plate)
    path_snap = directory + directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_realigned_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)


OSError: The file /scratch/shared/AMF914/Fiji.app/20201123_0421_Plate40/Analysis/skeleton_realigned_compressed.mat does not exist.

In [35]:
plt.close('all')
start = 20
finish = start + 
for i in range(start, finish):
    plot_t_tp1([],[],None,None,skels[i],skels[i+1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
begin = 0
end = 20
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
zone = (13000,13000+5000+3000,20000,20000+5000+4096)
skels_aligned = []
for i,date in enumerate(dates):
    directory_name=f'2020{dates[i]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skels_aligned.append(sio.loadmat(path_snap+'/Analysis/skeleton_realigned.mat')['skeleton'])
for i in range(11,13):
    plot_t_tp1([],[],None,None,skels_aligned[i][zone[0]:zone[1],zone[2]:zone[3]].todense(),skels_aligned[i+1][zone[0]:zone[1],zone[2]:zone[3]].todense())

In [20]:
plt.close('all')
zone = (6000,13000,12000,22000)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(skels_aligned[11][zone[0]:zone[1],zone[2]:zone[3]].todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Create graphs***

In [64]:
num_parallel = 5
time = '30:00'
args=[plate]
run_parallel('extract_nx_graph.py',args,begin,end,num_parallel,time,'extract_nx')

***Identify Nodes***

In [8]:
num_parallel = 1
time = '24:00:00'
args=[plate,begin,end]
run_parallel('extract_nodes_surf.py',args,0,0,num_parallel,time,'node_id')

***Check Node Id***

In [4]:
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen = dates_datetime[begin:end+1]
dates = dates_datetime_chosen
exp = Experiment(plate)
exp.load(dates)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/shared/AMF914/Fiji.app/20201123_1501_Plate39/Analysis/nx_graph_pruned_labeled.p'

In [55]:
plt.close('all')
t = 15
nodes = np.random.choice(exp.nx_graph[t].nodes, 100)
# exp.plot([8,13,14],[[794]]*3)
exp.plot([t,t+1,t+2],[nodes]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
exp.get_node(794).degree(12)

3

***Hyphae extraction***

In [56]:
num_parallel = 1
time = '2:00:00'
args = [plate, begin, end]
run_parallel('hyphae_extraction.py',args,0,0,num_parallel,time,'hyphae')

***Check Hyphae***

In [4]:
from directory import directory

dates_datetime = get_dates_datetime(directory,plate)
print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = dates_datetime
exp = pickle.load( open(f'{directory}Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

begin = 2020-11-19 20:21:00 
  end = 2020-11-23 00:21:00


FileNotFoundError: [Errno 2] No such file or directory: '/scratch/shared/AMF914/Fiji.app/Analysis_Plate40_2020-11-19 20:21:00_2021-01-10 11:50:00/experiment_40.pick'

In [17]:
hyph = choice(exp.hyphaes)
hyph.ts

[14, 15, 16, 17]

In [18]:
plt.close('all')
hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.close('all')
exp.plot([0,hyph.ts[-2],hyph.ts[-1]],[[hyph.root.label,hyph.end.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …